# Project : Signed Networks

## Milestone : P2 and P4
- Name : Emery  
- Surname : Sébastien 
- Sciper : 258565
- Groupe : Gaussiens 
- Paper : Signed Networks in Social Media (Leskovec, Huttemlocher and Kleinberg) (2010)
- Goal : Replication of Table 1 and 3 from the paper

# 1) Loading the data

We first import the libraries needed for the analysis of signed networks

In [1]:
# Libraries
import pandas as pd
import networkx as nx
import igraph
import numpy as np

#### The three following dataset will be  analyzed: 

1. Epinions : soc-sign-epinions.txt 
2. Sladshot : soc-sign-Slashdot090221.txt
3. Wikipedia : wikiElec.ElecBs3.txt

- The first two files' **columns** are structured as follows : **[FromNodeId, ToNodeId, Sign]**  and can be directly loaded as pandas dataframe using the tabular separator
- The last files has to be parsed manually to only gather the **useful informations** (nodes and edges)  

Load epinions as a datframe with **[FromNodeId, ToNodeId, Sign]** as columns. 

In [2]:
# Load as pandas dataframe
epinions = pd.read_csv("data/soc-sign-epinions.txt",header = 3, sep='\t')
# Remove the '#' sign from the column name
epinions.rename(columns={"# FromNodeId": "FromNodeId"}, inplace = True)
epinions

,FromNodeId,ToNodeId,Sign
0,0,1,-1
1,1,128552,-1
2,2,3,1
3,4,5,-1
4,4,155,-1
...,...,...,...
841367,131821,366,1
841368,131822,112101,1
841369,131823,131824,1
841370,131825,131826,1


Load sladshot as a datframe with **[FromNodeId, ToNodeId, Sign]** as columns. 

In [3]:
# Load as pandas dataframe
sladshot = pd.read_csv("data/soc-sign-Slashdot090221.txt",header = 3, sep='\t')
# Remove the '#' sign from the column name
sladshot.rename(columns={"# FromNodeId": "FromNodeId"}, inplace = True)
sladshot

,FromNodeId,ToNodeId,Sign
0,0,1,1
1,0,2,1
2,0,3,1
3,0,4,1
4,0,5,1
...,...,...,...
549197,82140,81612,1
549198,82141,82129,1
549199,82141,82142,1
549200,82143,81974,1


#### Parsing function

1. Open the the file using the right encoding and read the lines from it using the python in-build methods
2. Iterate over each row
3. Apart from the header, each row begins by a letter :
    - E: is election succesful (1) or not (0)
    - T: time election was closed
    - U: user id (and username) of editor that is being considered for promotion
    - N: user id (and username) of the nominator
    - V: <vote(1:support, 0:neutral, -1:oppose)> <user_id> <time> <username>

4. Only the lines with the user and voter's information are retain, because we are only interested in the voter-user information and not the temporal evolution yet for table 1. 
5. The file is structured as we have a user line first and all the voters associated after. 
6. Keep the user id in information when encounter a user line.
7. If the vote is not neutral (0) append to list a list composed of ['VoterId','UserId', 'Sign'] using the user in memory

In [4]:
def parse_txt_line(file_name): 
    """
    Input :  wikiElec.ElecBs3.txt.txt file to parse
    Output : A list containing lists of ['VoterId','UserId', 'Sign']
    """
    # output table
    from_to_sign=[]
    # open the file
    with open(file_name, 'r', encoding="Latin-1") as fp:
        # read the lines
        Lines = fp.readlines() 
        # user in memory
        user = ''
        for line in Lines: 
            # access a user line
            if line[0] == 'U':
                values_users = line.strip().split('\t')
                user = values_users[1]
            # access a voter line
            if line[0] == 'V':
                values_voters = line.strip().split('\t')
                # if the vote is not neutral add ['VoterId','UserId', 'Sign']
                if int(values_voters[1]) != 0 :
                    from_to_sign.append([int(values_voters[2]),int(user),int(values_voters[1])])
    return from_to_sign

In [5]:
# parse the file using the function a get the list
wikipedia_raw_data = parse_txt_line('data/wikiElec.ElecBs3.txt')
# Instantiate a dataframe using the list
wikipedia = pd.DataFrame(wikipedia_raw_data,columns=['FromNodeId', 'ToNodeId', 'Sign'])
wikipedia

,FromNodeId,ToNodeId,Sign
0,3,30,1
1,25,30,-1
2,4,30,1
3,5,30,1
4,6,30,1
...,...,...,...
107075,8045,6307,-1
107076,7053,6307,-1
107077,6885,6307,-1
107078,8243,6307,-1


# 2) Data Wrangling
In this section, we will observe several facts about the graphs and create multiples version of the edgelists, which can be compared to the statistics available.

#### Neutral links removal
The first part has already done during the parsing of the raw wikipedia file by removing the neutral votes. For the others two dataset, it either has already be done or it inherently doesn't have neutral link.

#### Duplicated links 
In the wikipedia dataset, a candidate can be presented multiples times until the election's result is positive. Therefore, multiples identical and opposite links can remain in the data. To count triangles without sign information we do not need them, as a consequence we create a filtered version for the triads analysis without duplicates.

In [6]:
# 1) Removal of identical duplicated links
# keep the last occurence of the any links duplicated or not -> filter every occurence of duplicated links except the last 
wikipedia_duplicated_identical = wikipedia[wikipedia.duplicated(keep='last') == False]
print('The number of identical duplicated links that has been removed is : {}'.format( len(wikipedia.index)-len(wikipedia_duplicated_identical.index)))
wikipedia_duplicated_identical

The number of identical duplicated links that has been removed is : 2721


,FromNodeId,ToNodeId,Sign
0,3,30,1
1,25,30,-1
2,4,30,1
3,5,30,1
4,6,30,1
...,...,...,...
107075,8045,6307,-1
107076,7053,6307,-1
107077,6885,6307,-1
107078,8243,6307,-1


In [7]:
# 2) Removal of opposite sign duplicated links
# keep the last occurence of the identical pair of (user,voters) independently of the sign it will result 
wikipedia_duplicated = wikipedia_duplicated_identical[wikipedia_duplicated_identical.duplicated(['FromNodeId','ToNodeId'],keep='last') == False]
print('The number of opposite sign duplicated links that has been removed is : {}'.format( len(wikipedia_duplicated_identical.index)-len(wikipedia_duplicated.index)))
# reset the index 
wikipedia_duplicated = wikipedia_duplicated.reset_index(drop=True)
wikipedia_duplicated

The number of opposite sign duplicated links that has been removed is : 612


,FromNodeId,ToNodeId,Sign
0,3,30,1
1,25,30,-1
2,4,30,1
3,5,30,1
4,6,30,1
...,...,...,...
103742,8045,6307,-1
103743,7053,6307,-1
103744,6885,6307,-1
103745,8243,6307,-1


#### Self connections
As for the duplicated links, self connections do not impact triangles and triads. However, it can impact the number of nodes and of course edges. I create a dataframe with self connections removed to be able to compare the statistics with https://snap.stanford.edu/data/wiki-Vote.html, which has removed them, for the triangle counting part. For, the sake of completness I also remove the self connections of epinions and sladshot does not have any.

In [8]:
# remove self connections by locating every pair that has different ids
epinions_self = epinions.loc[epinions['FromNodeId'] != epinions['ToNodeId']]
print('The number of self connections that has been removed is : {}'.format( len(epinions.index)-len(epinions_self.index)))

The number of self connections that has been removed is : 573


In [9]:
# remove self connections by locating every pair that has different ids
wikipedia_self_duplicated = wikipedia_duplicated.loc[wikipedia_duplicated['FromNodeId'] != wikipedia_duplicated['ToNodeId']]
print('The number of self connections that has been removed is : {}'.format( len(wikipedia_duplicated.index)-len(wikipedia_self_duplicated.index)))
# reset the index 
wikipedia_self_duplicated = wikipedia_self_duplicated.reset_index(drop=True)
wikipedia_self_duplicated

The number of self connections that has been removed is : 58


,FromNodeId,ToNodeId,Sign
0,3,30,1
1,25,30,-1
2,4,30,1
3,5,30,1
4,6,30,1
...,...,...,...
103684,8045,6307,-1
103685,7053,6307,-1
103686,6885,6307,-1
103687,8243,6307,-1


#### Summary of the different dataframe created and when they are used

|          Dataframe        |                Status                |              Usage                   |
|---------------------------|:------------------------------------:|-------------------------------------:|
|          epinions         |               raw data               | Triangle analysis,Table1 replication |
|          sladshot         |               raw data               | Triangle analysis,Table1 replication |
|          wikipedia        |           neutral filtered           |            Not used                  |
|    wikipedia_duplicated   |   neutral and duplicated filtered    |        Table1 replication            |
| wikipedia_self_duplicated | neutral,duplicated and self filtered |        Triangle analysis             |

# 3) Triangles analysis
In this section, we analyse the number of triangles present in each graph without sign and reciprocated edges information. As a matter of fact, reciprocated edges become one edge in undirected representation and without sign information, we do not need to worry about them. We will use the networkx library to construct the directed graph from our pandas dataframe as edges list and code our own function for triangle analysis using the undirected representation of the graph using the networkx function.

#### Graph instantiation

In [10]:
# epinions
G_epinions = nx.from_pandas_edgelist(epinions, 'FromNodeId', 'ToNodeId',["Sign"], create_using=nx.DiGraph()) 
print('Nodes: {n}, Edges: {e}'.format(n=nx.number_of_nodes(G_epinions), e=nx.number_of_edges(G_epinions)))

Nodes: 131828, Edges: 841372


In [11]:
# sladshot
G_sladshot = nx.from_pandas_edgelist(sladshot, 'FromNodeId', 'ToNodeId',["Sign"], create_using=nx.DiGraph()) 
print('Nodes: {n}, Edges: {e}'.format(n=nx.number_of_nodes(G_sladshot), e=nx.number_of_edges(G_sladshot)))

Nodes: 82140, Edges: 549202


**Use the wikipedia edges list without duplicated edges an self connection, as they do not impact the triangles counting and to be comparable with the statistics provided on the stanford website**

In [12]:
# wikipedia
G_wikipedia = nx.from_pandas_edgelist(wikipedia_self_duplicated, 'FromNodeId', 'ToNodeId',["Sign"], create_using=nx.DiGraph()) 
print('Nodes: {n}, Edges: {e}'.format(n=nx.number_of_nodes(G_wikipedia), e=nx.number_of_edges(G_wikipedia)))

Nodes: 7115, Edges: 103689


#### Triangles counting

The triangles counting is done by a custom function working on the dictionary representation of the graph. This is obtained using the _nx.to_dict_of_dicts()_ function that returns a dictionary with every nodes as principal keys and all the nodes connected to them as nested dictionnary linked to the main key. This avoid us to iterate on every three nodes combinations of the graph and saves us a lot of time. It works as follows :
1. Iterate over every nodes
2. For every nodes iterate over the keys inside the nested dictonnary (ie every connected node to the first)
3. For every second node connected to the first, iterate over its connected nodes
4. Check if the connnected nodes to the second is connected to the first
5. Sort the combination of three nodes so the order does not matter ( count only once no matter the combinations possible)
6. Save the result as a tuple of three nodes in a set, which can only have an element once in it
7. To have the number of triangle get the lenght of the ouptut set

**Only work for undirected representation of the graph as the connections is present in both nested dictionary for both nodes** <br>
Example :
1) 1->2 connection <br>
2) Corresponding directed dictionnary : {1:{2:{sign: }}} <br>
3) Corresponding undirected dictionnary : {1:{2:{sign: }},2:{1:{sign:}}}

In [13]:
def triangles(m):
    """
    A function that compute the number of triangle in an undirected graph
    
    Input : A sparse dictonnary representation of the graph
    Output : A set containing every tuples of vertices involved in a triangle
    """
    # ouptut set of tuples
    out = { 3: set()}
    # Iterate over every nodes in the graph
    for i, (n1, row) in enumerate(m.items()):
        # get all the connected nodes -> existing keys
        for n2 in row.keys():
            # iterate over row of connected node
            for n3 in m[n2]:
                # n1 exists in this row, all 3 nodes are connected to each other -> triangle
                if n3 in row:
                    if len({n1, n2, n3}) == 3:
                        t = tuple(sorted((n1, n2, n3)))
                        out[3].add(t)
    return out

In [14]:
%%time
# Epinions
# undirected representation
G_epinions_un = G_epinions.to_undirected()
# get the dictionary
Epinions_dict = nx.to_dict_of_dicts(G_epinions_un)
#compute triangles
epinions_triangles = triangles(Epinions_dict)
print('The number of triangles in epinions is : {}'.format(len(epinions_triangles[3])))

The number of triangles in epinions is : 4910076
Wall time: 1min 3s


In [15]:
%%time
# Sladshot
# undirected representation
G_sladshot_un = G_sladshot.to_undirected()
# get the dictionary
Sladshot_dict = nx.to_dict_of_dicts(G_sladshot_un)
#compute triangles
sladshot_triangles = triangles(Sladshot_dict)
print('The number of triangles in epinions is : {}'.format(len(sladshot_triangles[3])))

The number of triangles in epinions is : 579565
Wall time: 19.7 s


In [16]:
%%time
# wikipedia
# undirected representation
G_wikipedia_un = G_wikipedia.to_undirected()
# get the dictionary
Wikipedia_dict = nx.to_dict_of_dicts(G_wikipedia_un)
#compute triangles
wikipedia_triangles = triangles(Wikipedia_dict)
print('The number of triangles in epinions is : {}'.format(len(wikipedia_triangles[3])))

The number of triangles in epinions is : 608389
Wall time: 6.93 s


#### Results 

We summarize the results obtained in the following table :

|                        |    Epinions   |   Sladshot   |   Wikipedia   |
|------------------------|:-------------:|:------------:|--------------:|
|          Nodes         |    131828     |    82140     |     7115      |
|          Edges         |    841372     |    549202    |     103689    |
|        Triangles       |    4910076    |    579565    |     608389    |

Those results coincide with the statistics of the graph from the three dataset downloaded at the following URL :

1. https://snap.stanford.edu/data/soc-sign-epinions.html
2. http://snap.stanford.edu/data/soc-sign-Slashdot090221.html
3. https://snap.stanford.edu/data/wiki-Vote.html

Therefore, we were able to find every undirected triangle without sign information in the three graph. This is the first step to replicate Table 1, where the sign information is needed and it will be done in the next section.

# 4)  Replication of Table 1
In this section we try to replicate results from table 1 by incorporating the sign information of the edges. The first attempt is done using the triad_census function and speculating about the edges' sign. The second attempt wants to validate the first reasonning by finding the sign of the edges in the dataset during triangle counting to discriminate between signed triangles.

## A) Triad_census
The triadic census of a directed graph is summarized in the image below, but it is basically every possible way to connect three nodes with reciprocated edges possibles and no parrallel edges. There are sixteen types as explained in the following paper : [1] Vladimir Batagelj and Andrej Mrvar, A subquadratic triad census algorithm for large sparse networks with small maximum degree, University of Ljubljana
<br>
<img src="https://raw.githubusercontent.com/sebemery/ADA-CS401/main/SignedNetworks/images/triads_census.png" height="800" width="400">
<br>
<br>
Thus theoretically, the number of triangles found previously should matched the number of closed triads in the triadic census, as we are not taking into account signs and parrallel edges are forbidden. Moreover, we did not count reciprocated edges as two possibles triangles, therefore only counted them once. In the triadic census there are counted once also with taking into account that it is a reciprocated edges.

#### Triadic census matching triangles
In this analysis we use the _triad_census()_ function provided by the library igraph-python. The reason why we use this implementation rather than the networkx's one, is that it runs much faster (not shown here, but encountered during the process). This probably comes, that some of the functionality provided by the library are running in the C language, that is faster than python. The networkx's method is _O(m)_ with m the number of edges, the time complexity of the igraph's method is not available and I did not find the _triad_census_ C implementation, therefore we do not know the time complexity. However, it should not be much different form the networkx one and not impact a lot the difference in running time.

In [17]:
# epinions
# create a directed graph
g_epinions = igraph.Graph.DataFrame(epinions, directed=True)
print('Nodes: {n}, Edges: {e}'.format(n= g_epinions.vcount(), e=g_epinions.ecount()))

Nodes: 131828, Edges: 841372


In [18]:
# sladshot
# create a directed graph
g_sladshot = igraph.Graph.DataFrame(sladshot, directed=True)
print('Nodes: {n}, Edges: {e}'.format(n= g_sladshot.vcount(), e=g_sladshot.ecount()))

Nodes: 82140, Edges: 549202


**Use the wikipedia_duplicated edges list, as the statistics or nodes and edges matches the statistics of Table 1 in the paper, therefore, they probably worked with this data**

In [19]:
# wikipedia
# create a directed graph
g_wikipedia = igraph.Graph.DataFrame(wikipedia_duplicated, directed=True)
print('Nodes: {n}, Edges: {e}'.format(n= g_wikipedia.vcount(), e=g_wikipedia.ecount()))

Nodes: 7118, Edges: 103747


In [20]:
%%time
# triadic_census for epinions
tc_epinions = g_epinions.triad_census()

Wall time: 2min 53s


In [21]:
%%time
# triadic_census for sladshot
tc_sladshot = g_sladshot.triad_census()

Wall time: 27.8 s


In [22]:
%%time
# triadic_census for wikipedia
tc_wikipedia = g_wikipedia.triad_census()

Wall time: 7.26 s


The _triad_census()_ method return a dictionary with the keys corresponding to the label on the image diplayed above and the number associated. Therefore to count the number of triangles we just access the keys that are closed triads and summed them.

In [23]:
# triangles counting
triangles_epinions = tc_epinions['030T'] + tc_epinions['030C'] + tc_epinions['120D'] + tc_epinions['120U'] + tc_epinions['120C'] + tc_epinions['210'] + tc_epinions['300']
triangles_sladshot = tc_sladshot['030T'] + tc_sladshot['030C'] + tc_sladshot['120D'] + tc_sladshot['120U'] + tc_sladshot['120C'] + tc_sladshot['210'] + tc_sladshot['300']
triangless_wikipedia = tc_wikipedia['030T'] + tc_wikipedia['030C'] + tc_wikipedia['120D'] + tc_wikipedia['120U'] + tc_wikipedia['120C'] + tc_wikipedia['210'] + tc_wikipedia['300']
print('The number of triangles with the triad_census method for epinions is : {n}'.format(n= triangles_epinions))
print('The number of triangles with  the triad_census method for sladshot is : {n}'.format(n= triangles_sladshot))
print('The number of triangles with  the triad_census method for wikipedia is : {n}'.format(n= triangless_wikipedia))

The number of triangles with the triad_census method for epinions is : 4910076
The number of triangles with  the triad_census method for sladshot is : 579565
The number of triangles with  the triad_census method for wikipedia is : 608389


As expected we get the same number of triangles as the method in the previous part even for wikipedia as we just add self connections which do not impact triangles. This was done to justify the next reasonning.

#### Triadic census for signed triangles
As the number of triangles is the same for both, we can see that reciprocated edges are counted as one single edges without directionality in the case of triangles. However, if we want to take the sign into account, those reciprocated edges becomes one, but the sign could be either 1 or -1 depending on the sign of the reciprocated edges. Thus, to differentiate signed triangles, we can speculate and say that in the data the reciprocated edges have the opposite sign. Therefore, when counting the sign we need to multiply by two the number of unsigned triangles for each reciprocated edges in it to get the two signed version one with 1 and one with -1 for the previously reciprocated edges. 

In [24]:
# signed triangles counting
triads_epinions = tc_epinions['030T'] + tc_epinions['030C'] + 2*tc_epinions['120D'] + 2*tc_epinions['120U'] + 2*tc_epinions['120C'] + 4*tc_epinions['210'] + 8*tc_epinions['300']
triads_sladshot = tc_sladshot['030T'] + tc_sladshot['030C'] + 2*tc_sladshot['120D'] + 2*tc_sladshot['120U'] + 2*tc_sladshot['120C'] + 4*tc_sladshot['210'] + 8*tc_sladshot['300']
triads_wikipedia = tc_wikipedia['030T'] + tc_wikipedia['030C'] + 2*tc_wikipedia['120D'] + 2*tc_wikipedia['120U'] + 2*tc_wikipedia['120C'] + 4*tc_wikipedia['210'] + 8*tc_wikipedia['300']
print('The number of triads with sign taking into account for epinions is : {n}'.format(n= triads_epinions))
print('The number of triads with sign taking into account for sladshot is : {n}'.format(n= triads_sladshot))
print('The number of triads with sign taking into account for wikipedia is : {n}'.format(n= triads_wikipedia))

The number of triads with sign taking into account for epinions is : 13317672
The number of triads with sign taking into account for sladshot is : 1508105
The number of triads with sign taking into account for wikipedia is : 790532


Results shows that for both sladshot and wikipedia we get the same number of signed triads and a similar value for the epinions network.

#### Edges percentage 
Get the the percentage of each type of edges using the pandas edges list to count the occurence of both types 

In [25]:
# Get the number of edges type as a series with two values
epinions_sign = epinions['Sign'].value_counts()
sladshot_sign = sladshot['Sign'].value_counts()
wikipedia_sign = wikipedia_duplicated['Sign'].value_counts()

In [26]:
# Display the percentage as the occurence divide by the total
print('The epinions percentage of (+) edges is : {e1} and (-) edges is : {e2}'.format(e1= epinions_sign[1]/(epinions_sign[1]+epinions_sign[-1])*100,e2 =  epinions_sign[-1]/(epinions_sign[1]+epinions_sign[-1])*100  ))
print('The sladshot percentage of (+) edges is : {e1} and (-) edges is : {e2}'.format(e1= sladshot_sign[1]/(sladshot_sign[1]+sladshot_sign[-1])*100,e2 =  sladshot_sign[-1]/(sladshot_sign[1]+sladshot_sign[-1])*100  ))
print('The wikipedia percentage of (+) edges is : {e1} and (-) edges is : {e2}'.format(e1= wikipedia_sign[1]/(wikipedia_sign[1]+wikipedia_sign[-1])*100,e2 =  wikipedia_sign[-1]/(wikipedia_sign[1]+wikipedia_sign[-1])*100  ))

The epinions percentage of (+) edges is : 85.29722881198803 and (-) edges is : 14.702771188011962
The sladshot percentage of (+) edges is : 77.39811581166856 and (-) edges is : 22.601884188331432
The wikipedia percentage of (+) edges is : 78.7858926041235 and (-) edges is : 21.214107395876507


#### Results 

We summarize the results obtained in the following table :

|                        |    Epinions   |   Sladshot   |   Wikipedia   |
|------------------------|:-------------:|:------------:|--------------:|
|          Nodes         |    131828     |    82140     |     7118      |
|          Edges         |    841372     |    549202    |     103747    |
|        (+) Edges       |    85.3 %     |    77.4 %    |     78.79 %   |
|        (-) Edges       |    14.7 %     |    22.6 %    |     21.21 %   |
|          Triads        |   13317672    |   1508105    |     790532    |

#### Discussion
1. The sladshot network has four nodes less than in the paper. However, statistics from the standford websites is consistent with the number of nodes obtained. They either worked with a slighlty different dataset with four more nodes or they just took the highest id plus one to get this number. Indeed, the list of ids goes from 0 to 82143 with four ids missing : 11681, 73374, 78483 and 79626 (code not provided, trivial task)
2. The number of nodes and edges from the epinions network differ from the number in the paper, but the percentage remains close to the paper. No clear explanation was provided on data wrangling and no obvious processing as the removing the duplicated edges or self connections ( 573 edges removed and at best 573 nodes removed)  get us close to the paper's number, this is the best attempt we achieved.
3. The number of triangles is the same for the three dataset compared to the stanford websites and the number of signed triads is the same for sladshot and wikipedia compared to the paper. However, it is different for epinions (little bit less) , but in the same magnitudes. This comes probably from the slightly different version of the data.
4. All others numbers not mentionned previously are exactly replicated.

## B) Signed triangles
In this section, we try to access the access the sign of the reciprocated edges in the directed representation of the graph to confirm the results obtained in A) and not speculate.

#### Signed triangles counting
The principles of the custom function used are the following :
1. Uses the set of tuples containing the triplets of nodes containing the triangles (from part 3)
2. Iterate over each tuple/triangle
3. Signed triangles are constituted of one undirected edge between each nodes coming from either one directed edge or a reciprocated edges
4. For each triplet, there can be eight configurations with one directed edge between two nodes (one can draw them by hand to convince himself)
5. For the eight configurations of a triplet, try to get the sign of the directed edges in the directed dictionnary representation of the graph.
6. If the directed configuration of edges do not exist an exception is raised, because one or more edges are not present in the directed dictionary and we handle them by simply passing.
7. If the configuration do exist, sum the sign of the triangle's edges to discriminate between the four kind of signed triangles (see image below).
8. Store the signed triangle in the appropriated list in the output dictionnary.

<img src="https://raw.githubusercontent.com/sebemery/ADA-CS401/main/SignedNetworks/images/signed_triangles.PNG" height="800" width="400">

In [27]:
def signed_triangles(triplet,m_dir):
    """
    A function that compute the number of signed triangle given the triangle and the directed representation of the graph
    
    Input : A set of tuples constituting the triangles and a sparse dictonnary representation of the directed graph
    Output : A dictionary of each kind of signed triangles stored as lists containing the edges as tuples
    """
    # output dictionary of singed triangles stored in lists
    out = { 'T0': [], 'T1': [], 'T2': [], 'T3': []}
    # Iterate over every triplets of nodes constituting a triangle
    for i,t in enumerate(triplet): 
        # config 1 : a->b->c->a
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[0]][t[1]]['Sign'] + m_dir[t[1]][t[2]]['Sign'] + m_dir[t[2]][t[0]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[0],t[1])),tuple((t[1],t[2])),tuple((t[2],t[0]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
        # config 2 : a->c->b->a   
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[0]][t[2]]['Sign'] + m_dir[t[2]][t[1]]['Sign'] + m_dir[t[1]][t[0]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[0],t[2])),tuple((t[2],t[1])),tuple((t[1],t[0]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
        # config 3 : c<-a->b->c
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[0]][t[1]]['Sign'] + m_dir[t[0]][t[2]]['Sign'] + m_dir[t[1]][t[2]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[0],t[1])),tuple((t[0],t[2])),tuple((t[1],t[2]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
        # config 4 : c<-a->b<-c
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[0]][t[1]]['Sign'] + m_dir[t[0]][t[2]]['Sign'] + m_dir[t[2]][t[1]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[0],t[1])),tuple((t[0],t[2])),tuple((t[2],t[1]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
        # config 5 : c->a<-b->c  
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[1]][t[0]]['Sign'] + m_dir[t[2]][t[0]]['Sign'] + m_dir[t[1]][t[2]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[1],t[0])),tuple((t[2],t[0])),tuple((t[1],t[2]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
        # config 6 :   c->a<-b<-c  
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[1]][t[0]]['Sign'] + m_dir[t[2]][t[0]]['Sign'] + m_dir[t[2]][t[1]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[1],t[0])),tuple((t[2],t[0])),tuple((t[2],t[1]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
            
        # config 7  :c->a->b<-c
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[0]][t[1]]['Sign'] + m_dir[t[2]][t[0]]['Sign'] + m_dir[t[2]][t[1]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[0],t[1])),tuple((t[2],t[0])),tuple((t[2],t[1]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
            
        # config 8 : c<-a<-b->c
        try:
            # sum over the sign of the directed edges in the triangle
            sum_sign = m_dir[t[0]][t[2]]['Sign'] + m_dir[t[1]][t[0]]['Sign'] + m_dir[t[1]][t[2]]['Sign']
            # lists of tuples representing the directed edges with the first element pointing to the second element of the tuple
            t_signed = [tuple((t[0],t[2])),tuple((t[1],t[0])),tuple((t[1],t[2]))]
            if sum_sign == -3:
                out['T0'].append(t_signed)
            if sum_sign == -1:
                 out['T1'].append(t_signed)
            if sum_sign == 1:
                out['T2'].append(t_signed)
            if sum_sign == 3:
                out['T3'].append(t_signed)

        except KeyError :
            pass
    return out

In [28]:
%%time
# epinions
# construct the directed dictionary
Epinions_dict_dir = nx.to_dict_of_dicts(G_epinions)
# compute the signed triangles using the set of triplets from part 3
epinions_signed_triangles = signed_triangles(epinions_triangles[3],Epinions_dict_dir)
# sum over each signed triangle's type to get the overall number
epinions_total_signed_triangles = len(epinions_signed_triangles['T0'])+len(epinions_signed_triangles['T1'])+len(epinions_signed_triangles['T2'])+len(epinions_signed_triangles['T3'])
print('The number of signed triangles for epinions is  : {n}'.format(n=epinions_total_signed_triangles ))
print('The number of T0 signed triangles for epinions is  : {n}'.format(n=len(epinions_signed_triangles['T0']) ))
print('The number of T1 signed triangles for epinions is  : {n}'.format(n=len(epinions_signed_triangles['T1']) ))
print('The number of T2 signed triangles for epinions is  : {n}'.format(n=len(epinions_signed_triangles['T2']) ))
print('The number of T3 signed triangles for epinions is  : {n}'.format(n=len(epinions_signed_triangles['T3']) ))

The number of signed triangles for epinions is  : 13317672
The number of T0 signed triangles for epinions is  : 87668
The number of T1 signed triangles for epinions is  : 688557
The number of T2 signed triangles for epinions is  : 924739
The number of T3 signed triangles for epinions is  : 11616708
Wall time: 1min 19s


In [29]:
%%time
# sladshot
# construct the directed dictionary
Sladshot_dict_dir = nx.to_dict_of_dicts(G_sladshot)
# compute the signed triangles using the set of triplets from part 3
sladshot_signed_triangles = signed_triangles(sladshot_triangles[3],Sladshot_dict_dir)
# sum over each signed triangle's type to get the overall number
sladshot_total_signed_triangles = len(sladshot_signed_triangles['T0'])+len(sladshot_signed_triangles['T1'])+len(sladshot_signed_triangles['T2'])+len(sladshot_signed_triangles['T3'])
print('The number of signed triangles for sladshot is  : {n}'.format(n=sladshot_total_signed_triangles))
print('The number of T0 signed triangles for sladshot is  : {n}'.format(n=len(sladshot_signed_triangles['T0'])))
print('The number of T1 signed triangles for sladshot is  : {n}'.format(n=len(sladshot_signed_triangles['T1'])))
print('The number of T2 signed triangles for sladshot is  : {n}'.format(n=len(sladshot_signed_triangles['T2'])))
print('The number of T3 signed triangles for sladshot is  : {n}'.format(n=len(sladshot_signed_triangles['T3'])))

The number of signed triangles for sladshot is  : 1508105
The number of T0 signed triangles for sladshot is  : 16272
The number of T1 signed triangles for sladshot is  : 115884
The number of T2 signed triangles for sladshot is  : 109303
The number of T3 signed triangles for sladshot is  : 1266646
Wall time: 11.9 s


In [30]:
%%time
# wikipedia
# construct the directed dictionary
Wikipedia_dict_dir = nx.to_dict_of_dicts(G_wikipedia)
# compute the signed triangles using the set of triplets from part 3
wikipedia_signed_triangles = signed_triangles(wikipedia_triangles[3],Wikipedia_dict_dir)
# sum over each signed triangle's type to get the overall number
wikipedia_total_signed_triangles = len(wikipedia_signed_triangles['T0'])+len(wikipedia_signed_triangles['T1'])+len(wikipedia_signed_triangles['T2'])+len(wikipedia_signed_triangles['T3'])
print('The number of signed triangles for wikipedia is  : {n}'.format(n=wikipedia_total_signed_triangles ))
print('The number of T0 signed triangles for wikipedia is  : {n}'.format(n= len(wikipedia_signed_triangles['T0']) ))
print('The number of T1 signed triangles for wikipedia is  : {n}'.format(n= len(wikipedia_signed_triangles['T1']) ))
print('The number of T2 signed triangles for wikipedia is  : {n}'.format(n= len(wikipedia_signed_triangles['T2']) ))
print('The number of T3 signed triangles for wikipedia is  : {n}'.format(n= len(wikipedia_signed_triangles['T3']) ))

The number of signed triangles for wikipedia is  : 790532
The number of T0 signed triangles for wikipedia is  : 8479
The number of T1 signed triangles for wikipedia is  : 63425
The number of T2 signed triangles for wikipedia is  : 163328
The number of T3 signed triangles for wikipedia is  : 555300
Wall time: 5.06 s


We confirmed the results obtained in A) by obtaining the same values for all three networks. Thus, we do not need to repeat the observations made in the last point for the replication of the table, the statistics are the same.

#### Results 

We summarize the signed triangles' results obtained in the following table :

|                        |    Epinions   |   Sladshot   |   Wikipedia   |
|------------------------|:-------------:|:------------:|--------------:|
|        $T_{0}$ (---)   |    87668      |    16272     |     8479      |
|        $T_{1}$ (+--)   |    688557     |    115884    |     63425     |
|        $T_{2}$ (++-)   |   924739      |    109303    |     163328    |
|        $T_{3}$ (+++)   |   11616708    |   1266646    |     555300    |
|         Total          |   13317672    |   1508105    |     790532    |

We see that we obtain the same numbers for the different kind of signed triangle in epinions and wikipedia graphs. As expected, epinions differ also for each kind. Thus, it reinforce the fact that they could have worked with a slightly different dataset or they performed a non obvious preprocessing step that we weren't able to reproduce given the elements in the paper. Nontheless, we were able to reproduce table 1 pretty accurately.

# 5) Replication of Table 3
In this section, we will try to replicate table 3. We re-use the signed triangle implementation which is exactly what the table is about. We found the distribution of signed triangles in the last section and we will re-use the results obtained in this part. First, we use these results to obtain the percentage of each kind present in the graphs and denote them as $p(T_{i})$. Then, we compare this empirical frequency of triangles to frequencies obtained from the same background of positive and negative edges, but assigned at random. Finally, we compute a surprise score for the triads to measure how significant the difference is. Thus, we shuflle the sign of each network and recompute the triangle's frequency which we called $p_{0}(T_{i})$.

#### A) Empirical triads frequency
In this subsection, we compute the percentage of each kind of triangles by dividing the specific number by the overall number. We re-use the results obtained in the last subsection signed triangles.

In [31]:
def compute_fraction(output, total):
    """
    Take as input the dictionary result from the last part : ouptut
    Take the total number of signed triangle of the last part : total
    
    Output : dataframe containing the distribution of signed triangle and another one containing the fraction
    """
    
    output_dict = { 'T0': [len(output['T0'])], 'T1': [len(output['T1'])], 'T2': [len(output['T2'])], 'T3': [len(output['T3'])]}
    data = pd.DataFrame.from_dict(output_dict, orient='columns')
    fraction = data / total
    return data,fraction

In [32]:
data_epinions,fraction_epinions = compute_fraction(epinions_signed_triangles, epinions_total_signed_triangles)
print('The fraction of T0 signed triangles for epinions is  : {n:.3f}'.format(n= fraction_epinions['T0'].values[0] ))
print('The fraction of T1 signed triangles for epinions is  : {n:.3f}'.format(n= fraction_epinions['T1'].values[0] ))
print('The fraction of T2 signed triangles for epinions is  : {n:.3f}'.format(n= fraction_epinions['T2'].values[0] ))
print('The fraction of T3 signed triangles for epinions is  : {n:.3f}'.format(n= fraction_epinions['T3'].values[0] ))

The fraction of T0 signed triangles for epinions is  : 0.007
The fraction of T1 signed triangles for epinions is  : 0.052
The fraction of T2 signed triangles for epinions is  : 0.069
The fraction of T3 signed triangles for epinions is  : 0.872


In [33]:
data_sladshot,fraction_sladshot = compute_fraction(sladshot_signed_triangles, sladshot_total_signed_triangles)
print('The fraction of T0 signed triangles for sladshot is  : {n:.3f}'.format(n= fraction_sladshot['T0'].values[0] ))
print('The fraction of T1 signed triangles for sladshot is  : {n:.3f}'.format(n= fraction_sladshot['T1'].values[0] ))
print('The fraction of T2 signed triangles for sladshot is  : {n:.3f}'.format(n= fraction_sladshot['T2'].values[0] ))
print('The fraction of T3 signed triangles for sladshot is  : {n:.3f}'.format(n= fraction_sladshot['T3'].values[0] ))

The fraction of T0 signed triangles for sladshot is  : 0.011
The fraction of T1 signed triangles for sladshot is  : 0.077
The fraction of T2 signed triangles for sladshot is  : 0.072
The fraction of T3 signed triangles for sladshot is  : 0.840


In [34]:
data_wikipedia,fraction_wikipedia = compute_fraction(wikipedia_signed_triangles, wikipedia_total_signed_triangles)
print('The fraction of T0 signed triangles for wikipedia is  : {n:.3f}'.format(n= fraction_wikipedia['T0'].values[0] ))
print('The fraction of T1 signed triangles for wikipedia is  : {n:.3f}'.format(n= fraction_wikipedia['T1'].values[0] ))
print('The fraction of T2 signed triangles for wikipedia is  : {n:.3f}'.format(n= fraction_wikipedia['T2'].values[0] ))
print('The fraction of T3 signed triangles for wikipedia is  : {n:.3f}'.format(n= fraction_wikipedia['T3'].values[0] ))

The fraction of T0 signed triangles for wikipedia is  : 0.011
The fraction of T1 signed triangles for wikipedia is  : 0.080
The fraction of T2 signed triangles for wikipedia is  : 0.207
The fraction of T3 signed triangles for wikipedia is  : 0.702


#### B) Random frequency and surprise score
In this subsection, we shuffle the sign of the edges in the network using the sample function of pandas, thus creating a new edge_list. We create multiple shuffled edges list using different seed and average it. Then, the a priori probability of triad $T_{i}$ is computed by dividing the number of specific triad by the overall and it is called $p_{0}(T_{i})$ . Then we say that a type of triads is overrepresented relative to the chance (random) if $p(T_{i})>p_{0}(T_{i})$. To measure how significatif the difference is, we compute the suprise  as described in the paper: $s(T_{i}) = \frac{T_{i}-E[T_{i}]}{\sqrt{\Delta \cdot p_{0}(T_{i}) \cdot(1-p_{0}(T_{i}))}}$. This represent the number of standard deviations by which the empirical number of triads $T_{i}$ differs from the expected number under the random-shuffling model which tend to a normal law by the central limit theorem. This is already very significant for surprise on the order of ten.

In [35]:
def shuffle_signs_triangles(random_states,graph):
    """
    A function that creates multiples shuffled version of the graph taken as input and ouptut a dictionnary with the
    distribution of the signed triangles for each of them
    
    Input : 1) list of seeds to shuffle the edge list
            2) edge_list (dataframe) of the graph
    
    Output : Dictionnary of the signed triangles distribution for each shuffled version
    """
    # output dictionary containing the distribution of signed triangles for multiples random states
    out = { 'T0': [], 'T1': [], 'T2': [], 'T3': []}
    
    # iterate over the random states
    for seed in random_states:
        
        # shuffle the signs
        sign = graph.drop(['FromNodeId','ToNodeId'], axis=1)
        sign = sign.sample(frac=1, random_state=seed)
        sign.reset_index(inplace=True, drop=True)
        graph_shuffled = graph.copy()
        graph_shuffled['Sign'] = sign['Sign']
        
        # create a graph
        G_graph_shuffled = nx.from_pandas_edgelist(graph_shuffled, 'FromNodeId', 'ToNodeId',["Sign"], create_using=nx.DiGraph())
        
        # compute triangles
        # undirected representation
        G_graph_s_un = G_graph_shuffled.to_undirected()
        # get the dictionary
        graph_s_dict = nx.to_dict_of_dicts(G_graph_s_un)
        #compute triangles
        graph_s_triangles = triangles(graph_s_dict)
        
        # compute signed triangles
        # construct the directed dictionary
        graph_dict_dir = nx.to_dict_of_dicts(G_graph_shuffled)
        # compute the signed triangles using the set of triplets from part 3
        graph_signed_triangles = signed_triangles(graph_s_triangles[3],graph_dict_dir)
        
        # output dictionnary
        out['T0'].append(len(graph_signed_triangles['T0']))
        out['T1'].append(len(graph_signed_triangles['T1']))
        out['T2'].append(len(graph_signed_triangles['T2']))
        out['T3'].append(len(graph_signed_triangles['T3']))
    return out

In [36]:
def compute_stats(data, output_shuffle, delta):
    """
    Compute the prior probability and the surprise for each kind of signed triangle by averaging the shuffled 
    version statistics
    
    Input : 1) dictionnary of the empirical distribution
            2) dictionnary of the shuffled version distribution
            3) Total number of triangles in the network
            
    Output : 1) dataframe containing the prior probability of each kind
             2) dataframe containing the surprise of each kind of triangle
    """
    data_shuffle = pd.DataFrame.from_dict(output_shuffle, orient='columns')
    means = pd.Series(data_shuffle.mean())
    p_0 = means/delta
    data = data.append(p_0,ignore_index=True)
    s =  data.apply(lambda x: (x[0]-delta*x[1])/np.sqrt(((delta*x[1])*(1-x[1]))), axis=0)
    return p_0,s

In [37]:
out_epinions = shuffle_signs_triangles([0,1,2,3,4,5,6,7,8,9], epinions)

In [38]:
out_sladshot = shuffle_signs_triangles([0,1,2,3,4,5,6,7,8,9], sladshot)

In [39]:
out_wikipedia = shuffle_signs_triangles([0,1,2,3,4,5,6,7,8,9], wikipedia_duplicated)

In [40]:
p_0_epinions ,s_epinions= compute_stats(data_epinions,out_epinions,epinions_total_signed_triangles)
print('The a priori probability of T0 signed triangles for epinions is : {n:.3f}'.format(n= p_0_epinions['T0'] ))
print('The a priori probability of T1 signed triangles for epinions is : {n:.3f}'.format(n= p_0_epinions['T1'] ))
print('The a priori probability of T2 signed triangles for epinions is : {n:.3f}'.format(n= p_0_epinions['T2'] ))
print('The a priori probability of T3 signed triangles for epinions is : {n:.3f}'.format(n= p_0_epinions['T3'] ))
print('The surprise of T0 signed triangles for epinions is : {n:.3f}'.format(n= s_epinions['T0'] ))
print('The surprise of T1 signed triangles for epinions is : {n:.3f}'.format(n= s_epinions['T1'] ))
print('The surprise of T2 signed triangles for epinions is : {n:.3f}'.format(n= s_epinions['T2'] ))
print('The surprise of T3 signed triangles for epinions is : {n:.3f}'.format(n= s_epinions['T3']))

The a priori probability of T0 signed triangles for epinions is : 0.003
The a priori probability of T1 signed triangles for epinions is : 0.055
The a priori probability of T2 signed triangles for epinions is : 0.321
The a priori probability of T3 signed triangles for epinions is : 0.621
The surprise of T0 signed triangles for epinions is : 221.155
The surprise of T1 signed triangles for epinions is : -55.056
The surprise of T2 signed triangles for epinions is : -1964.611
The surprise of T3 signed triangles for epinions is : 1890.378


In [44]:
p_0_sladshot ,s_sladshot= compute_stats(data_sladshot,out_sladshot,sladshot_total_signed_triangles)
print('The a priori probability of T0 signed triangles for sladshot is : {n:.3f}'.format(n= p_0_sladshot['T0'] ))
print('The a priori probability of T1 signed triangles for sladshot is : {n:.3f}'.format(n= p_0_sladshot['T1'] ))
print('The a priori probability of T2 signed triangles for sladshot is : {n:.3f}'.format(n= p_0_sladshot['T2'] ))
print('The a priori probability of T3 signed triangles for sladshot is : {n:.3f}'.format(n= p_0_sladshot['T3'] ))
print('The surprise of T0 signed triangles for sladshot is : {n:.3f}'.format(n= s_sladshot['T0'] ))
print('The surprise of T1 signed triangles for sladshot is : {n:.3f}'.format(n= s_sladshot['T1'] ))
print('The surprise of T2 signed triangles for sladshot is : {n:.3f}'.format(n= s_sladshot['T2'] ))
print('The surprise of T3 signed triangles for sladshot is : {n:.3f}'.format(n= s_sladshot['T3']))

The a priori probability of T0 signed triangles for sladshot is : 0.011
The a priori probability of T1 signed triangles for sladshot is : 0.118
The a priori probability of T2 signed triangles for sladshot is : 0.406
The a priori probability of T3 signed triangles for sladshot is : 0.465
The surprise of T0 signed triangles for sladshot is : -7.431
The surprise of T1 signed triangles for sladshot is : -156.348
The surprise of T2 signed triangles for sladshot is : -833.837
The surprise of T3 signed triangles for sladshot is : 923.656


In [42]:
p_0_wikipedia ,s_wikipedia= compute_stats(data_wikipedia,out_wikipedia,wikipedia_total_signed_triangles)
print('The a priori probability of T0 signed triangles for wikipedia is : {n:.3f}'.format(n= p_0_wikipedia['T0'] ))
print('The a priori probability of T1 signed triangles for wikipedia is : {n:.3f}'.format(n= p_0_wikipedia['T1'] ))
print('The a priori probability of T2 signed triangles for wikipedia is : {n:.3f}'.format(n= p_0_wikipedia['T2'] ))
print('The a priori probability of T3 signed triangles for wikipedia is : {n:.3f}'.format(n= p_0_wikipedia['T3'] ))
print('The surprise of T0 signed triangles for wikipedia is : {n:.3f}'.format(n= s_wikipedia['T0'] ))
print('The surprise of T1 signed triangles for wikipedia is : {n:.3f}'.format(n= s_wikipedia['T1'] ))
print('The surprise of T2 signed triangles for wikipedia is : {n:.3f}'.format(n= s_wikipedia['T2'] ))
print('The surprise of T3 signed triangles for wikipedia is : {n:.3f}'.format(n= s_wikipedia['T3']))

The a priori probability of T0 signed triangles for wikipedia is : 0.010
The a priori probability of T1 signed triangles for wikipedia is : 0.107
The a priori probability of T2 signed triangles for wikipedia is : 0.395
The a priori probability of T3 signed triangles for wikipedia is : 0.489
The surprise of T0 signed triangles for wikipedia is : 10.098
The surprise of T1 signed triangles for wikipedia is : -76.138
The surprise of T2 signed triangles for wikipedia is : -342.893
The surprise of T3 signed triangles for wikipedia is : 380.403


#### Results 

We summarize the signed triangles' results obtained in the following table' :

|       Epinions        |   $T_{i}$     |  $p(T_{i})$  |   $p_{0} $     |  $s(T_{i})$ |  
|-----------------------|:-------------:|-------------:|----------------|------------:|
|       $T_{0}$ (---)   |    87668      |    0.007     |     0.003      |   221.155   |
|       $T_{1}$ (+--)   |    688557     |    0.052     |     0.055      |   -55.056   |
|       $T_{2}$ (++-)   |    924739     |    0.069     |     0.321      |   -1964.611 |
|       $T_{3}$ (+++)   |   11616708    |    0.872     |     0.621      |   1890.378  |


|       Sladshot        |   $T_{i}$     |  $p(T_{i})$  |   $p_{0} $   |  $s(T_{i})$ |  
|-----------------------|:-------------:|-------------:|----------------|------------:|
|       $T_{0}$ (---)   |    16272      |    0.011     |     0.011      |   -7.431    |
|       $T_{1}$ (+--)   |    115884     |    0.077     |     0.118      |   -156.348  |
|       $T_{2}$ (++-)   |    109303     |    0.072     |     0.406      |   -833.837  |
|       $T_{3}$ (+++)   |   1266646     |    0.840     |     0.465      |   923.656   |



|       Wikipedia       |   $T_{i}$     |  $p(T_{i})$  |   $p_{0} $     |  $s(T_{i})$ |  
|-----------------------|:-------------:|-------------:|----------------|------------:|
|       $T_{0}$ (---)   |    8479       |    0.011     |     0.010      |   10.098    |
|       $T_{1}$ (+--)   |    63425      |    0.080     |     0.107      |   -76.138   |
|       $T_{2}$ (++-)   |    163328     |    0.207     |     0.395      |   -342.893  |
|       $T_{3}$ (+++)   |   555300      |    0.702     |     0.489      |   380.403   |


#### Observation:
1. For all statistics, the numbers for $T_{1}$ and $T_{2}$ triads are inverted compared to the paper's number. However, in the case of epinions and sladshot the value found are exactly the same and after discussion with other students, they also found this inversion. Thus, the inversion is likely due to the authors and we won't mentionned it below.
2. As already mentionned in the previous step, for both epinions and sladshot we foind the exact same number of triads of each type. For the wikipedia dataset, we find value in the same order of magnitude . Thus, we succesfully managed to find the number of each triads in each dataset.
3. The empirical and prior probabilities of the triads follow the same observation as stated for the triads number. However, results are accurate to the thousandth which is negligeable.
4. Surprise value for triads $T_{1}$ and $T_{2}$ are in the same order of magnitudes compared to the paper. However, $T_{1}$ and $T_{2}$ surprise value seems to differ from the paper for epinions and wikipedia and have the same order of magnitude for sladshot.

#### Discussion:
1. All $T_{3}$ triads are overrepresented compared to the chance which fits Heider's theory
2. All $T_{2}$ triads are underrepresented compared to the chance which fits Heider's theory
3. $T_{1}$ triads are slightly overrepresented compared to the chance which fits more Davis's weaker notion of balance
4. $T_{0}$ triads are slightly underrepresented compared to the chance which fits more Davis's weaker notion of balance

In conclusion, we were able to reproduce qualitatively the result of the paper and draw approximately the same conclusion, even if the numercial results differ a little bit.